In [ ]:
import cv2
import os

data_path = 'Datasets'
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]

label_dict = dict(zip(categories, labels))

In [19]:
img_size = 32
data = []
target = []

facedata = "haarcascade_frontalface_default.xml"
cascade = cv2.CascadeClassifier(facedata)


for category in categories:
    folder_path = os.path.join(data_path, category)
    img_names = os.listdir(folder_path)
        
    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        faces = cascade.detectMultiScale(img)
        try:
            for f in faces:
                x, y, w, h = [v for v in f]
                sub_face = img[y:y + h, x:x + w]
                gray = cv2.cvtColor(sub_face, cv2.COLOR_BGR2GRAY)           
                resized = cv2.resize(gray, (img_size, img_size))
                data.append(resized)
                target.append(label_dict[category])
        except Exception as e:
            print('Exception:', e)

In [20]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
data = np.array(data) / 255.0
data = np.reshape(data, (data.shape[0], img_size, img_size, 1))
target = np.array(target)

from keras.utils import np_utils

new_target = np_utils.to_categorical(target)

np.save('./training/data', data)
np.save('./training/target', new_target)

In [21]:
import numpy as np

data=np.load('./training/data.npy')
target=np.load('./training/target.npy')

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint

noOfFilters = 64
sizeOfFilter1=(3, 3)
sizeOfFilter2=(3, 3)
sizeOfPool = (2, 2)
noOfNode = 64

model=Sequential()

#2 layere convolutionale 2D cu 32 de filtre care extrag caracteristici din datele de intrare
model.add((Conv2D(32, sizeOfFilter1, input_shape = data.shape[1:], activation = 'relu')))
model.add((Conv2D(32, sizeOfFilter1, activation = 'relu')))
model.add(MaxPooling2D(pool_size = sizeOfPool))

#2 layere convolutionale 2D cu 64 de filtre care extrag caracteristici de nivel superior din rezultatul celor cu 32 de filtre
model.add((Conv2D(64, sizeOfFilter2, activation = 'relu')))
model.add((Conv2D(64, sizeOfFilter2, activation = 'relu')))
model.add(MaxPooling2D(pool_size = sizeOfPool))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(noOfNode, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [23]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(data, target, test_size = 0.1)

In [24]:
checkpoint = ModelCheckpoint('./training/model-{epoch:03d}.model', monitor = 'val_loss', verbose = 0, save_best_only = True, mode = 'auto')
history = model.fit(train_data, train_target, epochs = 20, callbacks = [checkpoint], validation_split = 0.2)

Epoch 1/20
570/571 [============================>.] - ETA: 0s - loss: 0.5456 - accuracy: 0.7326

INFO:tensorflow:Assets written to: ./training\model-001.model\assets


INFO:tensorflow:Assets written to: ./training\model-001.model\assets


571/571 [==============================] - 38s 64ms/step - loss: 0.5452 - accuracy: 0.7328 - val_loss: 0.3600 - val_accuracy: 0.8392
Epoch 2/20
570/571 [============================>.] - ETA: 0s - loss: 0.3602 - accuracy: 0.8479

INFO:tensorflow:Assets written to: ./training\model-002.model\assets


...

INFO:tensorflow:Assets written to: ./training\model-015.model\assets


571/571 [==============================] - 26s 46ms/step - loss: 0.1707 - accuracy: 0.9324 - val_loss: 0.2123 - val_accuracy: 0.9181
Epoch 16/20
571/571 [==============================] - ETA: 0s - loss: 0.1644 - accuracy: 0.9331

INFO:tensorflow:Assets written to: ./training\model-016.model\assets


INFO:tensorflow:Assets written to: ./training\model-016.model\assets


571/571 [==============================] - 26s 46ms/step - loss: 0.1644 - accuracy: 0.9331 - val_loss: 0.2085 - val_accuracy: 0.9198
Epoch 17/20
571/571 [==============================] - 25s 44ms/step - loss: 0.1561 - accuracy: 0.9370 - val_loss: 0.2140 - val_accuracy: 0.9165
Epoch 18/20
571/571 [==============================] - 25s 44ms/step - loss: 0.1527 - accuracy: 0.9390 - val_loss: 0.2129 - val_accuracy: 0.9198
Epoch 19/20
571/571 [==============================] - 25s 44ms/step - loss: 0.1511 - accuracy: 0.9406 - val_loss: 0.2294 - val_accuracy: 0.9133
Epoch 20/20
571/571 [==============================] - 25s 44ms/step - loss: 0.1415 - accuracy: 0.9451 - val_loss: 0.2178 - val_accuracy: 0.9185


In [2]:
from keras.models import load_model
import cv2
import numpy as np

In [3]:
model = load_model('./training/model-005.model')

face_clsfr = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)

labels_dict = {0:'Male', 1:'Female'}
color_dict = {0:(0, 0, 255), 1:(0, 255, 0)}

In [5]:
while(True):

    #ret,img = cap.read()
    #img = cv2.imread("face1.jpg", cv2.IMREAD_COLOR)
    #img = cv2.imread("face2.png", cv2.IMREAD_COLOR)
    img = cv2.imread("faces.jpg", cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_clsfr.detectMultiScale(gray, 1.3, 3)  

    for (x,y,w,h) in faces:
        # extrage fetele din frame-uri; schimba dimensiunea fetelor la 32x32 pixeli;
        # normalizeaza valuarea pixelilor sa fie intre 0 si 1
        # remodeleaza imaginea intr-un tensor 4D; transmite tensorul la modleul CNN pentru a primi o predictie
        # extrage label-ul cu cea mai mare probabilitate din predicitii 

        face_img = gray[y:y+w, x:x+w]
        resized = cv2.resize(face_img, (32, 32))
        normalized = resized / 255.0
        reshaped = np.reshape(normalized,(1, 32, 32, 1))
        result = model.predict(reshaped)

        label = np.argmax(result, axis = 1)[0]
      
        cv2.rectangle(img, (x, y), (x + w, y + h), color_dict[label], 2)
        cv2.rectangle(img, (x, y - 40), (x + w, y), color_dict[label], -1)
        cv2.putText(img, labels_dict[label], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        
    cv2.imshow('Result', img)
    k = cv2.waitKey(1)
    
    if k == ord("q"):
        break
        
cv2.destroyAllWindows()
cap.release()

1/1 [==============================] - 0s 18ms/step
